In [1]:
import random
import numpy as np

from RobotEnviroment.init import *
from RobotEnviroment.arenas import RectangularArena
from PointClouds.InteractionPointFinder import getPushPoints
from PointClouds.Registration import *
from PathPlanning.scanning import *
from PathPlanning.motion import *
from utils.debug import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Initialise Robot and Configuration

In [2]:
ON_REAL = True
ROBOT_VELOCITY = .3 # rad/s
INITIAL_OBJ_POS = np.array([-.50, -.1, .69])

allowedViewAngleSegments = [[0., np.pi*.25], [.75*np.pi, 1.25*np.pi], [1.75*np.pi, np.pi*2]]

C = setup_config(INITIAL_OBJ_POS, ON_REAL)
bot = startup_robot(C, ON_REAL)
visualizeViewAreas(C, INITIAL_OBJ_POS, allowedViewAngleSegments)

-- bot.cpp:BotOp:44(0) CONNECTING TO FRANKAS
-- franka.cpp:init:34(0) FRANKA: Kp_freq:[20, 20, 20, 20, 10, 15, 10] Kd_ratio:[0.6, 0.6, 0.4, 0.4, 0.1, 0.5, 0.1] friction:[0, 0, 0, 0, 0, 0, 0]
-- franka.cpp:init:46(0) launching Franka 0 at 172.16.0.2
-- FrankaGripper.cpp:FrankaGripper:14(0) launching FrankaGripper 0 at 172.16.0.2
-- FrankaGripper.cpp:FrankaGripper:18(0) gripper max width:0.0793621


Define an arena to stop the robot from moving the object too far

In [3]:
TABLE_CENTER = np.array([-.23, -.16, .651])
TABLE_DIMS = np.array([.89, .55])
ROBOT_POS = np.array([-.03, -.24, .651])
ROBOT_RING = .29

pushArena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0], height=TABLE_DIMS[1], middlePCirc=ROBOT_POS, innerR=ROBOT_RING, name="pushArena")
pushArena.plotArena(C, color=[.5, .5, .5])

scanArena = RectangularArena(middleP=TABLE_CENTER, width=TABLE_DIMS[0]+.2, height=TABLE_DIMS[1]+.2, middlePCirc=ROBOT_POS, innerR=ROBOT_RING, name="scanArena")
scanArena.plotArena(C, color=[1., 1., 1.])

### Main pushing loop
- Look towards the predicted object position
- Get the object's point cloud and calculate its normals
- Choose a random pushing point in the point cloud
- Calculate the pushing motion
- Push if the motion is feasible

In [4]:
NUMBER_OF_PUSH_TRIALS = 10
predObjPos = INITIAL_OBJ_POS

pointClouds = [] # Stores the world position of the point cloud and the point cloud {"world_position": [x, y, z], "pc": np.array([])}
minNumScans = 2 # Minimum nuber of point cloud scans until we start merging them
fullPC = np.array([])
maxForces = []

for i in range(NUMBER_OF_PUSH_TRIALS):

    print("Starting Trial Number ", i+1)

    # Scan object at it's predicted position and get it's possible push points
    print("Scanning object...")
    lookAngle = giveRandomAllowedAngle(allowedViewAngleSegments)
    lookAtObjFromAngle(predObjPos, bot, C, lookAngle, velocity=ROBOT_VELOCITY, verbose=0)
    predObjPos, pointCloud = getScannedObject(bot, C, scanArena)
    if not len(predObjPos):
        print ("Lost the Object!")
        break
    push_points  = getPushPoints(pointCloud, verbose=1)

    # Store the object's point cloud
    print("Storing point cloud")
    for j, _ in enumerate(pointCloud):
        pointCloud[j] -= predObjPos
    pointClouds.append(pointCloud)

    if len(pointClouds) >= minNumScans:
        fullPC = joinOffsetPCS(pointClouds.copy(), verbose=2)

    # Try to push object
    for _ in range(5): # This counts the number of attempts to push an object from the current view angle. If it reaches 5 we try a new angle
        if not len(push_points):
            print("No viable push points found!")
            break

        pushP = random.choice(push_points)
        # push_points.remove(pushP)

        print("Trying to push obj...")
        waypoints = pushMotionWaypoints(pushP[0], pushP[1], predObjPos, config=C)
        if not pushArena.point_in_arena(waypoints[-1]): # Should automatically generate waypoints that are not outside the push arena but for now we do this
            print("Generated waypoints outside of arena!")
            continue

        success, maxForce = doPushThroughWaypoints(C, bot, waypoints, velocity=ROBOT_VELOCITY, verbose=0)
        if success:
            print("Success! :)")
            predObjPos = waypoints[-1] # This is offset by the width of the object!
            maxForces.append(maxForce)
            break
        print("Failed! :(")

Starting Trial Number  1
Scanning object...
0.3 0.157118
-- RealSenseThread.cpp:open:79(1) sensor Stereo Module
-- RealSenseThread.cpp:open:84(1)   option Exposure=8500  (Depth Exposure (usec))  [1,200000]
-- RealSenseThread.cpp:open:84(1)   option Gain=16  (UVC image gain)  [16,248]
-- RealSenseThread.cpp:open:84(1)   option Enable Auto Exposure=1  (Enable Auto Exposure)  [0,1]
-- RealSenseThread.cpp:open:84(1)   option Visual Preset=0  (Advanced-Mode Preset)  [0,5]
-- RealSenseThread.cpp:open:84(1)   option Laser Power=150  (Manual laser power in mw. applicable only when laser power mode is set to Manual)  [0,360]
-- RealSenseThread.cpp:open:84(1)   option Emitter Enabled=1  (Emitter select, 0-disable all emitters, 1-enable laser, 2-enable laser auto (opt), 3-enable LED (opt))  [0,2]
-- RealSenseThread.cpp:open:84(1)   option Frames Queue Size=16  (Max number of frames you can hold at a given time. Increasing this number will reduce frame drops but increase latency, and vice versa)  

In [ ]:
bot.home(C)

In [ ]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- FrankaGripper.cpp:~FrankaGripper:29(0) shutting down FrankaGripper
-- franka.cpp:~FrankaThread:14(0) shutting down Franka 0
-- franka.cpp:step:350(0) EXIT FRANKA CONTROL LOOP
-- RealSenseThread.cpp:~RealSenseThread:32(0) DTOR
-- RealSenseThread.cpp:close:161(0) STOPPING
